## DFT Course - 2022-23
## Beatriz Helena Cogollo-Olivo

## Tutorial 04 - Vibrations in solids

This example shows how to calculate vibrational properties for AlAs in zincblende structure.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Initial calculation

The first step is to perform a SCF calculation to obtain the wavefunctions that will be used in subsequent calculations.

In [ ]:
#Let's download the pseudopotentials
!wget http://pseudopotentials.quantum-espresso.org/upf_files/As.pz-n-rrkjus_psl.0.2.UPF
!wget http://pseudopotentials.quantum-espresso.org/upf_files/Al.pz-n-rrkjus_psl.0.1.UPF 

In [ ]:
%%writefile AlAs-SCF.in

&control
    calculation = 'scf',
    prefix = 'AlAs',
    wf_collect = .true. ,
    outdir = './tmp/',
    pseudo_dir = './',
 /
 &system
    ibrav =  2,
    celldm(1) = 10.50,
    nat =  2,
    ntyp = 2,
    ecutwfc = 40
 /
 &electrons
    mixing_beta = 0.6
 /
ATOMIC_SPECIES
 Al  26.98  Al.pz-n-rrkjus_psl.0.1.UPF
 As  74.92  As.pz-n-rrkjus_psl.0.2.UPF
ATOMIC_POSITIONS (alat)
 Al 0.00 0.00 0.00 
 As 0.25 0.25 0.25 
K_POINTS (automatic)
 4 4 4 0 0 0

In [ ]:
!pw.x < AlAs-SCF.in > AlAs-SCF.out

### Phonon calculation

Once we finished the SCF calculation, we can proceed with the phonon calculation. This calculation may take several minutes.

In [ ]:
%%writefile AlAs-PHONON.in

Phonons of AlAs
 &inputph
  tr2_ph=1.0d-12,
  prefix='AlAs',
  ldisp=.true.,
  nq1=4, 
  nq2=4, 
  nq3=4
  amass(1)=26.98,  !Mass of first atom
  amass(2)=74.92,  !Mass of second atom
  outdir='./tmp/',
  fildyn='AlAs-DynamicalMatrix.dyn',
 /

In [ ]:
!ph.x < AlAs-PHONON.in > AlAs-PHONON.out

In [ ]:
!grep -A N "Dynamical matrices" AlAs-PHONON.out  #Adjust N until all points are displayed

In [ ]:
!cat AlAs-DynamicalMatrix.dyn0

#### Question 1 
How many irreducible q points are calculated?

#### Question 2
Are the reported in the Phonon output file consistent with the Dynamical matrix file?

### IFC calculation
We can calculate the interatomic force constants (IFC) in real space by Fourier-transforming the dynamical matrices.

In [ ]:
%%writefile AlAs-Q2R.in
&input
   fildyn = 'AlAs-DynamicalMatrix.dyn', 
   zasr = 'simple', 
   flfrc = 'AlAs-Forces.fc'
 /
EOF

In [ ]:
!q2r.x < AlAs-Q2R.in > AlAs-Q2R.out

### Phonon dispersion calculation

In [ ]:
%%writefile AlAs-PHBands.in
&input
   asr = 'crystal' ,
   dos = .false. ,
   amass(1) = 26.98,
   amass(2) = 74.92,
   flfrc = 'AlAs-Forces.fc' ,
   flfrq = 'AlAs-Frequencies.freq' ,
   q_in_band_form = .true. ,
   q_in_cryst_coord = .true.
/
7
0.0000000000    0.0000000000    0.0000000000 50  !G
0.5000000000    0.0000000000    0.5000000000 50  !X
0.6250000000    0.2500000000    0.6250000000 50  !U
0.0000000000    0.0000000000    0.0000000000 50  !G
0.5000000000    0.5000000000    0.5000000000 50  !L
0.5000000000    0.2500000000    0.7500000000 50  !W
0.5000000000    0.0000000000    0.5000000000 1   !X

In [ ]:
!matdyn.x < AlAs-PHBands.in > AlAs-PHBands.out 

In [ ]:
!sed -n Np AlAs-Frequencies.freq.gp #Set N for each high-symmetry point according to the spacing defined above

In [ ]:
# load data
data = np.loadtxt('AlAs-Frequencies.freq.gp')
n_row, n_col = data.shape

k=data[:,0]

for band in range(1, n_col):
    plt.plot(k, data[:, band], linewidth=1, alpha=0.5, color='k')
plt.xlim(min(k), max(k))

# High symmetry k-points (check bands_pp.out)
plt.axvline(HIGH-SYMMETRY, linewidth=0.75, color='k', alpha=0.5) #Repeat for each high-symmetry point reported

# text labels
#List the high-symmetry points reported in the output file
plt.xticks(ticks= [HIGH-SYMMETRY,...], \
           labels=['HIGH-SYMMETRY',...])                #List the names of the high-symmetry points    
plt.ylabel("Frequency (cm-1)")
plt.show()

#### Question 3
Is the system dynamically stable?

### Phonon DOS calculation

In [ ]:
%%writefile AlAs-PHDOS.in
&input
   asr = 'crystal' ,
   dos = .true. ,
   amass(1) = 26.98,
   amass(2) = 74.92,
   flfrc = 'AlAs-Forces.fc' ,
   fldos = 'AlAs-PHDOS.dat' ,
   nk1 = Q , !Same as q point grid
   nk2 = Q ,
   nk3 = Q
/

In [ ]:
!matdyn.x < AlAs-PHDOS.in > AlAs-PHDOS.out

In [ ]:
# load data
frequency, phdos_tot, phdos_atom1, phdos_atom2 = np.loadtxt('AlAs-PHDOS.dat', unpack=True)

# make plot
plt.plot(frequency, phdos_atom1, linewidth=0.75, color='#006699', label='Al PhDOS')
plt.plot(frequency, phdos_atom2, linewidth=0.75, color='r', label='As PhDOS')
plt.plot(frequency, phdos_tot, linewidth=0.75, color='k', label='Total')
plt.yticks([])
plt.xlabel('Frequency (cm-1)')
plt.ylabel('Phonon DOS')
plt.show()

#### Question 4
Is this graph consistent with the phonon dispersion?